In [1]:
import os
import json
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from tqdm import tqdm

In [2]:
# Определение путей к данным
# CURRENT_DIR = os.path.dirname(os.path.abspath(__file__))
DATA_DIR = '/kaggle/input/colors/dataset_colors'
TRAIN_DATA_DIR = '/kaggle/input/colors/dataset_colors/train_data'
TEST_DATA_DIR = '/kaggle/input/colors/dataset_colors/test_data'
TRAIN_CSV = '/kaggle/input/colors/dataset_colors/train_data.csv'
TEST_CSV = '/kaggle/input/colors/dataset_colors/test_data.csv'

In [3]:
TRANSLIT_TO_RU = {
    'bezhevyi': 'бежевый',
    'belyi': 'белый',
    'biryuzovyi': 'бирюзовый',
    'bordovyi': 'бордовый',
    'goluboi': 'голубой',
    'zheltyi': 'желтый',
    'zelenyi': 'зеленый',
    'zolotoi': 'золотой',
    'korichnevyi': 'коричневый',
    'krasnyi': 'красный',
    'oranzhevyi': 'оранжевый',
    'raznocvetnyi': 'разноцветный',
    'rozovyi': 'розовый',
    'serebristyi': 'серебряный',
    'seryi': 'серый',
    'sinii': 'синий',
    'fioletovyi': 'фиолетовый',
    'chernyi': 'черный'
}

In [4]:
# Маппинг цветов
COLORS = {
    'бежевый': 'beige',
    'белый': 'white',
    'бирюзовый': 'turquoise',
    'бордовый': 'burgundy',
    'голубой': 'blue',
    'желтый': 'yellow',
    'зеленый': 'green',
    'золотой': 'gold',
    'коричневый': 'brown',
    'красный': 'red',
    'оранжевый': 'orange',
    'разноцветный': 'variegated',
    'розовый': 'pink',
    'серебряный': 'silver',
    'серый': 'gray',
    'синий': 'blue',
    'фиолетовый': 'purple',
    'черный': 'black'
}

CATEGORIES = ['одежда для девочек', 'столы', 'стулья', 'сумки']


In [5]:
class ProductDataset(Dataset):
    def __init__(self, df, img_dir, transform=None, is_test=False):
        self.df = df.reset_index(drop=True)  # Сбрасываем индексы после фильтрации
        self.img_dir = img_dir
        self.transform = transform
        self.is_test = is_test
        self.category_to_idx = {cat: idx for idx, cat in enumerate(CATEGORIES)}
        self.color_to_idx = {color: idx for idx, color in enumerate(COLORS.keys())}
        
        # Проверяем все пути к изображениям заранее
        self.valid_indices = []
        for idx in range(len(df)):
            img_path = os.path.join(self.img_dir, f"{df.iloc[idx]['id']}.jpg")
            if os.path.exists(img_path):
                self.valid_indices.append(idx)
        
        if len(self.valid_indices) == 0:
            raise ValueError(f"Не найдено ни одного изображения в директории {img_dir}")
        
        print(f"Найдено {len(self.valid_indices)} валидных изображений из {len(df)}")
        
    def __len__(self):
        return len(self.valid_indices)
    
    def __getitem__(self, idx):
        real_idx = self.valid_indices[idx]
        img_id = self.df.iloc[real_idx]['id']
        img_path = os.path.join(self.img_dir, f"{img_id}.jpg")
        
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            print(f"Ошибка при загрузке изображения {img_path}: {str(e)}")
            raise
        
        if self.transform:
            try:
                image = self.transform(image)
            except Exception as e:
                print(f"Ошибка при применении трансформации к {img_path}: {str(e)}")
                raise
            
        category = self.category_to_idx[self.df.iloc[real_idx]['category']]
        
        if not self.is_test:
            color_translit = self.df.iloc[real_idx]['target']
            color_ru = TRANSLIT_TO_RU[color_translit]
            color = self.color_to_idx[color_ru]
            return image, category, color
        
        return image, category, img_id

In [6]:
import torch
import torch.nn as nn
from transformers import AutoProcessor, AutoModel, AutoTokenizer, SiglipImageProcessor

class ColorClassifier(nn.Module):
    def __init__(self, num_colors, num_categories):
        super(ColorClassifier, self).__init__()
        
        ckpt = "google/siglip2-base-patch16-384"
        # Image embedding model
        self.image_processor = SiglipImageProcessor.from_pretrained(ckpt)
        self.image_model = AutoModel.from_pretrained(ckpt, device_map="auto")

        # Text embedding model for categories
        self.text_tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
        self.text_model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")

        # Classification head
        combined_features_size = 768 + 312  # Image embeddings (1152) + text embeddings (312)
        self.score = torch.nn.Sequential(
            torch.nn.Dropout(0.1),
            torch.nn.Linear(combined_features_size, combined_features_size // 2),
            torch.nn.Dropout(0.1),
            torch.nn.GELU(),
            torch.nn.Linear(combined_features_size // 2, num_colors),
        )

    def embed_image(self, images):
        inputs = self.image_processor(images=images, return_tensors="pt").to(self.image_model.device)
        with torch.no_grad():
            embeddings = self.image_model.get_image_features(**inputs)
        return embeddings

    def embed_text(self, categories):
        if isinstance(categories, torch.Tensor):  # Если это тензор, конвертируем в список строк
            categories = categories.tolist()
        
        if isinstance(categories, list):  # Если список, приводим к строковому виду
            categories = [str(cat) for cat in categories]
    
        t = self.text_tokenizer(categories, padding=True, truncation=True, return_tensors='pt')
        with torch.no_grad():
            model_output = self.text_model(**{k: v.to(self.text_model.device) for k, v in t.items()})
        embeddings = model_output.last_hidden_state[:, 0, :]
        embeddings = torch.nn.functional.normalize(embeddings)
        return embeddings


    def forward(self, images, categories):
        image_emb = self.embed_image(images)
        text_emb = self.embed_text(categories)

        # Combine image and text embeddings
        combined = torch.cat([image_emb, text_emb], dim=1)

        return self.score(combined)

In [7]:
import torch
import torch.nn as nn
from tqdm import tqdm
import copy

# Импортируем метрики из scikit-learn
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def train_model(
    model,
    train_loader,
    val_loader,
    criterion,
    optimizer,
    scheduler,
    num_epochs=25
):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Используется устройство: {device}")
    
    model = model.to(device)
    criterion = criterion.to(device)
    
    # Для отслеживания лучшей F1 и лучшей модели
    best_f1 = 0.0
    best_model_state = None
    best_epoch = 0
    
    # Для логирования динамики обучения
    # Можно расширять этот словарь под свои нужды
    history = {
        "train_loss": [],
        "val_loss": [],
        "val_precision": [],
        "val_recall": [],
        "val_f1": [],
        "val_accuracy": []
    }
    
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        
        # -- ТРЕНИРОВОЧНЫЙ ЦИКЛ --
        for images, categories, colors in tqdm(train_loader,
                                               desc=f"Epoch {epoch+1}/{num_epochs}",
                                               leave=False):
            images = images.to(device)
            categories = categories.to(device)
            colors = colors.to(device)
            
            optimizer.zero_grad()
            outputs = model(images, categories)
            loss = criterion(outputs, colors)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
        
        # Подсчёт среднего train loss за эпоху
        train_loss /= len(train_loader)
        
        # -- ВАЛИДАЦИЯ --
        model.eval()
        val_loss = 0.0
        val_preds = []
        val_true = []
        
        with torch.no_grad():
            for images, categories, colors in val_loader:
                images = images.to(device)
                categories = categories.to(device)
                colors = colors.to(device)
                
                outputs = model(images, categories)
                loss = criterion(outputs, colors)
                val_loss += loss.item()
                
                # Получаем предсказанные классы
                preds = torch.argmax(outputs, dim=1)
                
                # Сохраняем предсказания и истинные метки для метрик
                val_preds.extend(preds.cpu().numpy())
                val_true.extend(colors.cpu().numpy())
        
        # Подсчёт среднего val loss за эпоху
        val_loss /= len(val_loader)
        
        # -- МЕТРИКИ --
        precision, recall, f1, _ = precision_recall_fscore_support(
            val_true, val_preds, average='macro'
        )
        accuracy = accuracy_score(val_true, val_preds)
        
        # Сохраняем результаты в history
        history["train_loss"].append(train_loss)
        history["val_loss"].append(val_loss)
        history["val_precision"].append(precision)
        history["val_recall"].append(recall)
        history["val_f1"].append(f1)
        history["val_accuracy"].append(accuracy)
        
        # Выводим краткую статистику по эпохе
        print(
            f"Эпоха [{epoch+1}/{num_epochs}] | "
            f"Train Loss: {train_loss:.4f} | "
            f"Val Loss: {val_loss:.4f} | "
            f"Val Precision (macro): {precision:.4f} | "
            f"Val Recall (macro): {recall:.4f} | "
            f"Val Accuracy: {accuracy:.4f} | "
            f"Val F1 (macro): {f1:.4f}"
        )
        
        # Если F1 улучшилась, сохраняем "лучшую" модель
        if f1 > best_f1:
            best_f1 = f1
            best_model_state = copy.deepcopy(model.state_dict())
            best_epoch = epoch
            
            # Сохраняем чекпойнт
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_loss': val_loss,
                'precision': precision,
                'recall': recall,
                'f1_score': f1,
                'accuracy': accuracy,
                # Если нужно, сохраняйте и размеры словарей, и другие параметры
                # 'num_colors': len(COLORS),
                # 'num_categories': len(CATEGORIES)
            }, 'best_model.pth')
            print(f'Сохранена лучшая модель с F1: {f1:.4f} на эпохе {epoch+1}')
        
        # Делаем шаг шедулера (если требуется)
        scheduler.step()
        
    print(f"Лучшая F1: {best_f1:.4f} на эпохе {best_epoch+1}")
    return best_model_state, history

In [8]:
def load_model(model_path):
    """Загрузка сохраненной модели"""
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Файл модели не найден: {model_path}")
        
    checkpoint = torch.load(model_path)
    model = ColorClassifier(
        num_colors=checkpoint['num_colors'],
        num_categories=checkpoint['num_categories']
    )
    model.load_state_dict(checkpoint['model_state_dict'])
    return model

In [9]:
def predict(model, test_loader):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Используется устройство: {device}")
    
    model = model.to(device)
    model.eval()
    
    predictions = []
    ids = []
    
    color_list = list(COLORS.keys())
    
    with torch.no_grad():
        for images, categories, img_ids in test_loader:
            images = images.to(device)
            categories = categories.to(device)
            
            outputs = model(images, categories)
            probs = torch.softmax(outputs, dim=1).cpu().numpy()
            
            for img_id, prob in zip(img_ids, probs):
                pred_dict = {color: float(p) for color, p in zip(color_list, prob)}
                pred_color = color_list[np.argmax(prob)]
                
                predictions.append({
                    'id': img_id,
                    'predict_proba': json.dumps(pred_dict),
                    'predict_color': pred_color
                })
                
    return pd.DataFrame(predictions)

In [10]:
def main():
    required_paths = [
        TRAIN_CSV,
        TEST_CSV,
        TRAIN_DATA_DIR,
        TEST_DATA_DIR
    ]
    
    for path in required_paths:
        if not os.path.exists(path):
            raise FileNotFoundError(f"Не найден путь: {path}")
            
    print("Все необходимые файлы и директории найдены")
    
    train_df = pd.read_csv(TRAIN_CSV)
    test_df = pd.read_csv(TEST_CSV)
    
    print(f"Исходный размер тренировочного датасета: {len(train_df)}")
    print(f"Исходный размер тестового датасета: {len(test_df)}")
    
    def check_image_exists(row, data_dir):
        img_path = os.path.join(data_dir, f"{row['id']}.jpg")
        return os.path.exists(img_path)
    
    train_df = train_df[train_df.apply(lambda x: check_image_exists(x, TRAIN_DATA_DIR), axis=1)]
    test_df = test_df[test_df.apply(lambda x: check_image_exists(x, TEST_DATA_DIR), axis=1)]
    
    print(f"\nРазмер тренировочного датасета после фильтрации: {len(train_df)}")
    print(f"Размер тестового датасета после фильтрации: {len(test_df)}")
    
    unique_colors = train_df['target'].unique()
    print("\nУникальные цвета в данных:")
    print(unique_colors)
    
    unknown_colors = [color for color in unique_colors if color not in TRANSLIT_TO_RU]
    if unknown_colors:
        raise ValueError(f"Найдены неизвестные цвета: {unknown_colors}")
    
    print("Все цвета успешно маппятся")
    
    if len(train_df) == 0:
        raise ValueError("После фильтрации не осталось тренировочных данных!")
    if len(test_df) == 0:
        raise ValueError("После фильтрации не осталось тестовых данных!")
    
    train_transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.RandomCrop(224),
        transforms.ToTensor(),
    ])
    
    test_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])
    
    train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['target'])
    train_dataset = ProductDataset(train_df, TRAIN_DATA_DIR, transform=train_transform)
    val_dataset = ProductDataset(val_df, TRAIN_DATA_DIR, transform=test_transform)
    test_dataset = ProductDataset(test_df, TEST_DATA_DIR, transform=test_transform, is_test=True)
    
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Используется устройство: {device}")
    
    model_path = 'best_model.pth'
    if os.path.exists(model_path):
        print("Загружаем существующую модель...")
        model = load_model(model_path)
        model = model.to(device)
    else:
        print("Начинаем обучение новой модели...")
        model = ColorClassifier(len(COLORS), len(CATEGORIES))
        model = model.to(device)
        
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
        
        # --- ВАЖНО: теперь train_model возвращает два значения ---
        best_model_state, history = train_model(
            model, train_loader, val_loader, criterion, optimizer, scheduler
        )
        model.load_state_dict(best_model_state)
    
    print("Делаем предсказания...")
    predictions_df = predict(model, test_loader)
    predictions_df.to_csv('submission.csv', index=False)
    print("Готово! Результаты сохранены в submission.csv")

In [11]:
if __name__ == '__main__':
    main()

Все необходимые файлы и директории найдены
Исходный размер тренировочного датасета: 33303
Исходный размер тестового датасета: 1434

Размер тренировочного датасета после фильтрации: 33303
Размер тестового датасета после фильтрации: 344

Уникальные цвета в данных:
['zelenyi' 'chernyi' 'belyi' 'bordovyi' 'krasnyi' 'bezhevyi'
 'raznocvetnyi' 'rozovyi' 'serebristyi' 'korichnevyi' 'fioletovyi' 'seryi'
 'goluboi' 'oranzhevyi' 'sinii' 'biryuzovyi' 'zolotoi' 'zheltyi']
Все цвета успешно маппятся
Найдено 26642 валидных изображений из 26642
Найдено 6661 валидных изображений из 6661
Найдено 344 валидных изображений из 344
Используется устройство: cuda
Начинаем обучение новой модели...


tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/118M [00:00<?, ?B/s]

Используется устройство: cuda


Epoch 1/25:   0%|          | 0/833 [00:00<?, ?it/s]It looks like you are trying to rescale already rescaled images. If the input images have pixel values between 0 and 1, set `do_rescale=False` to avoid rescaling them again.


KeyboardInterrupt: 